In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn import preprocessing

In [3]:
# pip install ipynb

In [4]:
def remove_uninformative_attrs(df):
    newdf = df.drop(['id', 'track_name', 'album_uri', 'album_name', 'track_uri', \
                     'artist_name', 'album_release_date', 'album_img', 'key_mode'], axis=1)
    return newdf

In [5]:
def extract_year(dates):
    return [date[0:4] for date in dates]

In [6]:
def binarize_mode(modes):
    return [0 if mode == 'minor' else 1 for mode in modes]

In [7]:
def load_dataset(path='../dataset_generation/data/Music_Dataset.csv'):
    df = pd.DataFrame(pd.read_csv(path))
    df = remove_uninformative_attrs(df)
    df['album_release_year'] = extract_year(df['album_release_year'])
    df['mode'] = binarize_mode(df['mode'])
    return df

In [8]:
#df = load_dataset()
#df.head()

In [9]:
def scale_data(x_train, x_test):
    scaler = preprocessing.StandardScaler()
    scaler.fit(x_train)
    x_train_transformed = scaler.transform(x_train)
    x_test_transformed = scaler.transform(x_test)
    return (x_train_transformed, x_test_transformed)